In [1]:
from copy import deepcopy
import numpy as np
from sortedcontainers import SortedSet

# Solver

Rubik's cube has the following index convention:

![Rubik indices](images/face-indices.png)

Given a state in numpy array of dimension 6 x 3 x 3, the index corresponds to the face indicated above. The central face in the image is the front face.
Each block is then a 3 x 3 array that corresponds to the cells.

Each cell colour is indicated by an integer between 0 to 5 (inclusive).

The correspondences between indices and faces are:
* 0: face up (U)
* 1: face left (L)
* 2: face front (F)
* 3: face right (R)
* 4: face down (D)
* 5: face back (B)

Each action is one of `"L"`, `"L'"`, `"R"`, `"R'"`, `"F"`, `"F'"`, `"B"`, `"B'"`, `"U"`, `"U'"`, `"D"`, `"D'"`, where action without apostrophe `'` is clockwise and action with apostrophe `'` is anticlockwise. Clockwise direction is determined by the rotation of the face.

In [2]:
class Rubik:
    actions = ["L", "L'", "R", "R'", "F", "F'", "B", "B'", "U", "U'", "D", "D'"]
    
    def apply_action(state, action):
        """Applies the action on the state and returns a new state.
        Both `state` and returned state must be numpy array of 6 x 3 x 3 that represents a state of a Rubik's cube.
        Action must be one of "L", "L'", "R", "R'", "F", "F'", "B", "B'", "U", "U'", "D", "D'".

        First, deepcopy the initial state, and then call the respective functions that mutates the copied state.
        Then, return the copied state.
        """

        state = deepcopy(state)
        match action:
            case "L":
                return Rubik.rotate_left_clockwise(state)
            case "L'":
                return Rubik.rotate_left_anticlockwise(state)
            case "R":
                return Rubik.rotate_right_clockwise(state)
            case "R'":
                return Rubik.rotate_right_anticlockwise(state)
            case "F":
                return Rubik.rotate_front_clockwise(state)
            case "F'":
                return Rubik.rotate_front_anticlockwise(state)
            case "B":
                return Rubik.rotate_back_clockwise(state)
            case "B'":
                return Rubik.rotate_back_anticlockwise(state)
            case "U":
                return Rubik.rotate_up_clockwise(state)
            case "U'":
                return Rubik.rotate_up_anticlockwise(state)
            case "D":
                return Rubik.rotate_down_clockwise(state)
            case "D'":
                return Rubik.rotate_down_anticlockwise(state)
            case _:
                raise ValueError(f"Unrecognised action {action}")

    def rotate_face_clockwise(face):
        temp_corner = face[0][0]
        face[0][0] = face[2][0]
        face[2][0] = face[2][2]
        face[2][2] = face[0][2]
        face[0][2] = temp_corner

        temp_side = face[0][1]
        face[0][1] = face[1][0]
        face[1][0] = face[2][1]
        face[2][1] = face[1][2]
        face[1][2] = temp_side

    def rotate_face_anticlockwise(face):
        temp_corner = face[0][0]
        face[0][0] = face[0][2]
        face[0][2] = face[2][2]
        face[2][2] = face[2][0]
        face[2][0] = temp_corner

        temp_side = face[0][1]
        face[0][1] = face[1][2]
        face[1][2] = face[2][1]
        face[2][1] = face[1][0]
        face[1][0] = temp_side

    def rotate_left_clockwise(state):
        temp1, temp2, temp3 = state[0, :, 0]
        state[0, :, 0] = state[5, :, 0]
        state[5, :, 0] = state[4, :, 0]
        state[4, :, 0] = state[2, :, 0]
        state[2, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[1])
        return state

    def rotate_left_anticlockwise(state):
        temp1, temp2, temp3 = state[0, :, 0]
        state[0, :, 0] = state[2, :, 0]
        state[2, :, 0] = state[4, :, 0]
        state[4, :, 0] = state[5, :, 0]
        state[5, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[1])
        return state

    def rotate_right_clockwise(state):
        temp1, temp2, temp3 = state[0, :, 2]
        state[0, :, 2] = state[2, :, 2]
        state[2, :, 2] = state[4, :, 2]
        state[4, :, 2] = state[5, :, 2]
        state[5, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[3])
        return state

    def rotate_right_anticlockwise(state):
        temp1, temp2, temp3 = state[0, :, 2]
        state[0, :, 2] = state[5, :, 2]
        state[5, :, 2] = state[4, :, 2]
        state[4, :, 2] = state[2, :, 2]
        state[2, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[3])
        return state

    def rotate_front_clockwise(state):
        temp1, temp2, temp3 = state[0, 2, :]
        state[0, 2, :] = state[1, :, 2][::-1]
        state[1, :, 2][::-1] = state[4, 0, :][::-1]
        state[4, 0, :][::-1] = state[3, :, 0]
        state[3, :, 0] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[2])
        return state

    def rotate_front_anticlockwise(state):
        temp1, temp2, temp3 = state[0, 2, :]
        state[0, 2, :] = state[3, :, 0]
        state[3, :, 0] = state[4, 0, :][::-1]
        state[4, 0, :][::-1] = state[1, :, 2][::-1]
        state[1, :, 2][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[2])
        return state

    def rotate_back_clockwise(state):
        temp1, temp2, temp3 = state[0, 0, :]
        state[0, 0, :] = state[3, :, 2]
        state[3, :, 2] = state[4, 2, :][::-1]
        state[4, 2, :][::-1] = state[1, :, 0][::-1]
        state[1, :, 0][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[5])
        return state

    def rotate_back_anticlockwise(state):
        temp1, temp2, temp3 = state[0, 0, :]
        state[0, 0, :] = state[1, :, 0][::-1]
        state[1, :, 0][::-1] = state[4, 2, :][::-1]
        state[4, 2, :][::-1] = state[3, :, 2]
        state[3, :, 2] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[5])
        return state

    def rotate_up_clockwise(state):
        temp1, temp2, temp3 = state[1, 0, :]
        state[1, 0, :] = state[2, 0, :]
        state[2, 0, :] = state[3, 0, :]
        state[3, 0, :] = state[5, 2, :][::-1]
        state[5, 2, :][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[0])
        return state

    def rotate_up_anticlockwise(state):
        temp1, temp2, temp3 = state[1, 0, :]
        state[1, 0, :] = state[5, 2, :][::-1]
        state[5, 2, :][::-1] = state[3, 0, :]
        state[3, 0, :] = state[2, 0, :]
        state[2, 0, :] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[0])
        return state

    def rotate_down_clockwise(state):
        temp1, temp2, temp3 = state[1, 2, :]
        state[1, 2, :] = state[5, 0, :][::-1]
        state[5, 0, :][::-1] = state[3, 2, :]
        state[3, 2, :] = state[2, 2, :]
        state[2, 2, :] = temp1, temp2, temp3
        Rubik.rotate_face_clockwise(state[4])
        return state

    def rotate_down_anticlockwise(state):
        temp1, temp2, temp3 = state[1, 2, :]
        state[1, 2, :] = state[2, 2, :]
        state[2, 2, :] = state[3, 2, :]
        state[3, 2, :] = state[5, 0, :][::-1]
        state[5, 0, :][::-1] = temp1, temp2, temp3
        Rubik.rotate_face_anticlockwise(state[4])
        return state

    def turn_tuple(state):
        return tuple(state.reshape(54))

    def is_terminal(state):
        centers = state[:, 1, 1]
        return np.all(state == centers[:, None][:, None])

    def turn_numpy(state):
        return np.array(state).reshape((6, 3, 3))

In [3]:
test_face = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])

test_face_rotated_clockwise = deepcopy(test_face)
Rubik.rotate_face_clockwise(test_face_rotated_clockwise)
test_face_rotated_clockwise_expected = np.array([
    [7, 4, 1],
    [8, 5, 2],
    [9, 6, 3],
])
assert np.all(test_face_rotated_clockwise == test_face_rotated_clockwise_expected)

test_face_rotated_anticlockwise = deepcopy(test_face)
Rubik.rotate_face_anticlockwise(test_face_rotated_anticlockwise)
test_face_rotated_anticlockwise_expected = np.array([
    [3, 6, 9],
    [2, 5, 8],
    [1, 4, 7],
])
assert np.all(test_face_rotated_anticlockwise == test_face_rotated_anticlockwise_expected)

In [3]:
test_cube = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[11, 12, 13],
     [14, 15, 16],
     [17, 18, 19]],

    [[21, 22, 23],
     [24, 25, 26],
     [27, 28, 29]],

    [[31, 32, 33],
     [34, 35, 36],
     [37, 38, 39]],

    [[41, 42, 43],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [57, 58, 59]],
])

In [5]:
test_left_clockwise_expected = np.array([
    [[51, 2, 3],
     [54, 5, 6],
     [57, 8, 9]],

    [[17, 14, 11],
     [18, 15, 12],
     [19, 16, 13]],

    [[1, 22, 23],
     [4, 25, 26],
     [7, 28, 29]],

    [[31, 32, 33],
     [34, 35, 36],
     [37, 38, 39]],

    [[21, 42, 43],
     [24, 45, 46],
     [27, 48, 49]],

    [[41, 52, 53],
     [44, 55, 56],
     [47, 58, 59]]
])

test_left_clockwise = Rubik.rotate_left_clockwise(deepcopy(test_cube))
assert np.all(test_left_clockwise_expected == test_left_clockwise)

In [7]:
test_left_anticlockwise_expected = np.array([
    [[21, 2, 3],
     [24, 5, 6],
     [27, 8, 9]],

    [[13, 16, 19],
     [12, 15, 18],
     [11, 14, 17]],

    [[41, 22, 23],
     [44, 25, 26],
     [47, 28, 29]],

    [[31, 32, 33],
     [34, 35, 36],
     [37, 38, 39]],

    [[51, 42, 43],
     [54, 45, 46],
     [57, 48, 49]],

    [[1, 52, 53],
     [4, 55, 56],
     [7, 58, 59]],
])

test_left_anticlockwise = Rubik.rotate_left_anticlockwise(deepcopy(test_cube))
assert np.all(test_left_anticlockwise_expected == test_left_anticlockwise)

In [4]:
test_right_clockwise_expected = np.array([
    [[1, 2, 23],
     [4, 5, 26],
     [7, 8, 29]],

    [[11, 12, 13],
     [14, 15, 16],
     [17, 18, 19]],

    [[21, 22, 43],
     [24, 25, 46],
     [27, 28, 49]],

    [[37, 34, 31],
     [38, 35, 32],
     [39, 36, 33]],

    [[41, 42, 53],
     [44, 45, 56],
     [47, 48, 59]],

    [[51, 52, 3],
     [54, 55, 6],
     [57, 58, 9]],
])

test_right_clockwise = Rubik.rotate_right_clockwise(deepcopy(test_cube))
assert np.all(test_right_clockwise_expected == test_right_clockwise)

In [5]:
test_right_anticlockwise_expected = np.array([
    [[1, 2, 53],
     [4, 5, 56],
     [7, 8, 59]],

    [[11, 12, 13],
     [14, 15, 16],
     [17, 18, 19]],

    [[21, 22, 3],
     [24, 25, 6],
     [27, 28, 9]],

    [[33, 36, 39],
     [32, 35, 38],
     [31, 34, 37]],

    [[41, 42, 23],
     [44, 45, 26],
     [47, 48, 29]],

    [[51, 52, 43],
     [54, 55, 46],
     [57, 58, 49]],
])

test_right_anticlockwise = Rubik.rotate_right_anticlockwise(deepcopy(test_cube))
assert np.all(test_right_anticlockwise_expected == test_right_anticlockwise)

In [6]:
test_front_clockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [19, 16, 13]],

    [[11, 12, 41],
     [14, 15, 42],
     [17, 18, 43]],

    [[27, 24, 21],
     [28, 25, 22],
     [29, 26, 23]],

    [[7, 32, 33],
     [8, 35, 36],
     [9, 38, 39]],

    [[37, 34, 31],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [57, 58, 59]],
])

test_front_clockwise = Rubik.rotate_front_clockwise(deepcopy(test_cube))
assert np.all(test_front_clockwise == test_front_clockwise_expected)

In [7]:
test_front_anticlockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [31, 34, 37]],

    [[11, 12, 9],
     [14, 15, 8],
     [17, 18, 7]],

    [[23, 26, 29],
     [22, 25, 28],
     [21, 24, 27]],

    [[43, 32, 33],
     [42, 35, 36],
     [41, 38, 39]],

    [[13, 16, 19],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [57, 58, 59]],
])

test_front_anticlockwise = Rubik.rotate_front_anticlockwise(deepcopy(test_cube))
assert np.all(test_front_anticlockwise == test_front_anticlockwise_expected)

In [8]:
test_back_clockwise_expected = np.array([
    [[33, 36, 39],
     [4, 5, 6],
     [7, 8, 9]],

    [[3, 12, 13],
     [2, 15, 16],
     [1, 18, 19]],

    [[21, 22, 23],
     [24, 25, 26],
     [27, 28, 29]],

    [[31, 32, 49],
     [34, 35, 48],
     [37, 38, 47]],

    [[41, 42, 43],
     [44, 45, 46],
     [11, 14, 17]],

    [[57, 54, 51],
     [58, 55, 52],
     [59, 56, 53]],
])

test_back_clockwise = Rubik.rotate_back_clockwise(deepcopy(test_cube))
assert np.all(test_back_clockwise == test_back_clockwise_expected)

In [15]:
test_back_anticlockwise_expected = np.array([
    [[17, 14, 11],
     [4, 5, 6],
     [7, 8, 9]],

    [[47, 12, 13],
     [48, 15, 16],
     [49, 18, 19]],

    [[21, 22, 23],
     [24, 25, 26],
     [27, 28, 29]],

    [[31, 32, 1],
     [34, 35, 2],
     [37, 38, 3]],

    [[41, 42, 43],
     [44, 45, 46],
     [39, 36, 33]],

    [[53, 56, 59],
     [52, 55, 58],
     [51, 54, 57]],
])

test_back_anticlockwise = Rubik.rotate_back_anticlockwise(deepcopy(test_cube))
assert np.all(test_back_anticlockwise == test_back_anticlockwise_expected)

In [10]:
test_up_clockwise_expected = np.array([
    [[7, 4, 1],
     [8, 5, 2],
     [9, 6, 3]],

    [[21, 22, 23],
     [14, 15, 16],
     [17, 18, 19]],

    [[31, 32, 33],
     [24, 25, 26],
     [27, 28, 29]],

    [[59, 58, 57],
     [34, 35, 36],
     [37, 38, 39]],

    [[41, 42, 43],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [13, 12, 11]],
])

test_up_clockwise = Rubik.rotate_up_clockwise(deepcopy(test_cube))
assert np.all(test_up_clockwise == test_up_clockwise_expected)

In [18]:
test_up_anticlockwise_expected = np.array([
    [[3, 6, 9],
     [2, 5, 8],
     [1, 4, 7]],

    [[59, 58, 57],
     [14, 15, 16],
     [17, 18, 19]],

    [[11, 12, 13],
     [24, 25, 26],
     [27, 28, 29]],

    [[21, 22, 23],
     [34, 35, 36],
     [37, 38, 39]],

    [[41, 42, 43],
     [44, 45, 46],
     [47, 48, 49]],

    [[51, 52, 53],
     [54, 55, 56],
     [33, 32, 31]],
])

test_up_anticlockwise = Rubik.rotate_up_anticlockwise(deepcopy(test_cube))
assert np.all(test_up_anticlockwise == test_up_anticlockwise_expected)

In [12]:
test_down_clockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[11, 12, 13],
     [14, 15, 16],
     [53, 52, 51]],

    [[21, 22, 23],
     [24, 25, 26],
     [17, 18, 19]],

    [[31, 32, 33],
     [34, 35, 36],
     [27, 28, 29]],

    [[47, 44, 41],
     [48, 45, 42],
     [49, 46, 43]],

    [[39, 38, 37],
     [54, 55, 56],
     [57, 58, 59]],
])

test_down_clockwise = Rubik.rotate_down_clockwise(deepcopy(test_cube))
assert np.all(test_down_clockwise == test_down_clockwise_expected)

In [22]:
test_down_anticlockwise_expected = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[11, 12, 13],
     [14, 15, 16],
     [27, 28, 29]],

    [[21, 22, 23],
     [24, 25, 26],
     [37, 38, 39]],

    [[31, 32, 33],
     [34, 35, 36],
     [53, 52, 51]],

    [[43, 46, 49],
     [42, 45, 48],
     [41, 44, 47]],

    [[19, 18, 17],
     [54, 55, 56],
     [57, 58, 59]],
])

test_down_anticlockwise = Rubik.rotate_down_anticlockwise(deepcopy(test_cube))
assert np.all(test_down_anticlockwise == test_down_anticlockwise_expected)

In [12]:
test_tuple_expected = (
    1, 2, 3, 4, 5, 6, 7, 8, 9,
    11, 12, 13, 14, 15, 16, 17, 18, 19,
    21, 22, 23, 24, 25, 26, 27, 28, 29,
    31, 32, 33, 34, 35, 36, 37, 38, 39,
    41, 42, 43, 44, 45, 46, 47, 48, 49,
    51, 52, 53, 54, 55, 56, 57, 58, 59,
)

test_tuple = Rubik.turn_tuple(deepcopy(test_cube))
assert test_tuple == test_tuple_expected

assert np.all(Rubik.turn_numpy(test_tuple_expected) == test_cube)

In [28]:
assert not Rubik.is_terminal(deepcopy(test_cube))

test_cube_finished = np.array([
    [[1, 1, 1],
     [1, 1, 1],
     [1, 1, 1]],

    [[2, 2, 2],
     [2, 2, 2],
     [2, 2, 2]],

    [[4, 4, 4],
     [4, 4, 4],
     [4, 4, 4]],

    [[5, 5, 5],
     [5, 5, 5],
     [5, 5, 5]],

    [[3, 3, 3],
     [3, 3, 3],
     [3, 3, 3]],

    [[6, 6, 6],
     [6, 6, 6],
     [6, 6, 6]],
])

test_cube_unfinished = np.array([
    [[1, 1, 1],
     [1, 1, 1],
     [1, 1, 1]],

    [[2, 2, 2],
     [2, 2, 2],
     [2, 2, 2]],

    [[4, 4, 4],
     [4, 4, 4],
     [4, 4, 4]],

    [[5, 5, 5],
     [5, 5, 5],
     [5, 5, 5]],

    [[3, 3, 3],
     [3, 3, 3],
     [3, 3, 3]],

    [[6, 6, 6],
     [6, 6, 6],
     [6, 6, 1]],
])

assert Rubik.is_terminal(test_cube_finished)
assert not Rubik.is_terminal(test_cube_unfinished)

The heuristic determines how good a state is.

It takes in a state of a numpy array 6 x 3 x 3, and the number of steps taken to reach that state.

It produces an numerical value, indicating how good a state is.
The higher the value of the heuristic, the better the state is, and hence the closer to the front it is in the search frontier.

Current approach: count the number of pairs of adjacent squares that have the same color.
Two squares are considered adjacent if they are on the same face and share a side border.
The heuristic is the sum of the number of such pairs and the number of steps taken.

In [4]:
def heuristic(state, steps_taken):
    """Returns the value that represent how close the given state is to the terminal state
    and how far the state is to the initial state.
    This value is to be used in a frontier that sorts states based on how good the state is.

    A state is a Rubik's cube state, which is a 6 x 3 x 3 np.array.
    Each 3 x 3 block is a Rubik's cube face. The configuration of the Rubik's cube can be found at the start.

    The higher the value of the heuristic, the closer it is to the terminal state.
    """
    def face_heuristic(face):
        count = 0
        for i in range(3):
            for j in range(3):
                if i < 2 and face[i][j] == face[i + 1][j]:
                    count += 1
                if j < 2 and face[i][j] == face[i][j + 1]:
                    count += 1
        return count

    return sum([face_heuristic(face) for face in state]) - 4 * steps_taken

In [22]:
test_h_cube = np.array([
    [[1, 1, 2],
     [2, 1, 3],
     [1, 4, 5]]
])

assert heuristic(test_h_cube, 0) == 2

In [23]:
test_h_cube = np.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 1]]
])

assert heuristic(test_h_cube, 0) == 0

In [24]:
test_h_cube = np.array([
    [[1, 2, 1],
     [4, 5, 1],
     [7, 8, 2]],
])

assert heuristic(test_h_cube, 0) == 1

In [25]:
test_h_cube = np.array([
    [[1, 2, 2],
     [3, 2, 2],
     [5, 6, 7]],
])

assert heuristic(test_h_cube, 0) == 4

In [26]:
test_h_cube = np.array([
    [[1, 1, 1],
     [2, 1, 2],
     [3, 4, 5]],

    [[1, 4, 4],
     [2, 4, 4],
     [3, 5, 6]],
])

assert heuristic(test_h_cube, 0) == 7

Solving the Rubik's cube involves informed search. This means we need a frontier and a set of explored states.

The set of explored states is a hashmap for O(1) check of membership and O(1) insertion. This can be done using the built-in `set`.

The frontier can be done using a combination of a balanced binary search tree and a hashmap, for O(log(n)) insertion, O(1) check for membership, O(log(n)) deletion while supporting comparison between two elements. The reason for the need of comparison is that, we will use a heuristic to determine how good a state is. For each new state, we will calculate the heuristic and use it to determine its position in the frontier. Hence the frontier needs to be able to support comparison between elements and insert a new element at any position.

We also needs each state to be comparable to each other. Since tuples cannot be directly compared to one another, we use a container `Node` that contains the state as a field, and is comparable to other `Node`s. We can then implement the comparison logic with the heuristic.

The class `Node` is also used for traceback logic one the terminal state is found.

For class `Node`, we need the following information to be stored:
* `state`: the state that it contains, which is the 54 squares on the Rubik's cube.
* `parent`: the parent node of the node. This is used for backtracking.
* `action`: the action that transitions from the state in the parent node to this state.
* `steps_taken`: the number of steps taken to reach this state. This is used for calculation of heuristic.

Since the numpy array of 6 x 3 x 3 is needed for Rubik's operation
and flattened 54-element tuple to be stored for calculation of explored states,
we can store the state as the flattened 54-element tuple, 
and use appropriate function to transform the state to the numpy array.

Note that only `state` is needed to determine equality of two nodes, for purpose of the set of explored state.

In [5]:
class Node:
    def __init__(self, state, steps_taken=0, parent=None, action=None):
        """Initialises a node.
        `state`: A numpy array of dimension 6 x 3 x 3
        `steps_taken`: The number of steps taken from the initial state.
        `parent`: An instance of `Node` that contains the previous state.
        `action`: The action that transitions the previous state to this state.
        """
        self.state = Rubik.turn_tuple(state)
        self.steps_taken = steps_taken
        self.parent = parent
        self.action = action
        self.h_value = heuristic(state, steps_taken)

    def __lt__(self, node):
        """Determines the priority of this node compared to another node.
        This determines the position of this node in the search frontier.
        """
        return self.h_value < node.h_value

    def __hash__(self):
        """Determines the hash value of this node.
        This enables the hashing algorithm of the set of explored states.
        Nodes with the same states must have the same hash values.
        """
        return hash(self.state)

    def __eq__(self, other):
        """Determines the equality of two nodes.
        This enables the hashing algorithm of the set of explored states.
        Since we do not want to re-explore the same states, any two nodes
        with the same state must be equal, regardless of the values of other fields.
        """
        return self.state == other.state
        
    def get_numpy_state(self):
        """Get the numpy array that represents the state contained in this node.
        The numpy array must be of dimension 6 x 3 x 3.
        """
        return Rubik.turn_numpy(self.state)

In [37]:
assert Node(test_cube) == Node(test_cube)
assert hash(Node(test_cube)) == hash(Node(test_cube))

assert Node(deepcopy(test_cube)) == Node(test_cube)
assert hash(Node(deepcopy(test_cube))) == hash(Node(test_cube))

test_random_cube = np.array([
    [[1, 4, 5],
     [2, 3, 7],
     [1, 2, 3]],

    [[1, 5, 6],
     [2, 4, 1],
     [2, 4, 1]],

    [[1, 4, 6],
     [6, 3, 4],
     [4, 2, 5]],

    [[2, 3, 1],
     [2, 3, 4],
     [1, 6, 5]],

    [[5, 3, 4],
     [1, 4, 2],
     [4, 3, 1]],

    [[4, 5, 6],
     [1, 2, 4],
     [3, 2, 1]],
])

assert not Node(test_random_cube) == Node(test_cube)

In [38]:
assert np.all(Node(deepcopy(test_cube)).get_numpy_state() == test_cube)

In [39]:
assert Node(test_cube) < Node(test_random_cube)

The search algorithm is an informed search that involves:

* `explored_states`: Keep track of explored states. This would be the set of nodes. Note that equality of nodes is equality of the contained states.
* `frontier`: The list of nodes, ordered by their heuristic. Make use of `SortedSet`.

The terminal state can be determined before child nodes are inserted to the `frontier`.
We can then make use of backtracking, which relies on the `parent` and the `action` fields of the nodes.

In [6]:
def solve(state):
    """Solve the Rubik's cube from the initial state.
    The state is a numpy array of 6 x 3 x 3.

    We make use of `Node`, which stores the necessary information of efficient search and backtracking,
    `Rubik` for Rubik's cube logic, and `SortedSet` for the frontier.
    """

    first_node = Node(state)
    frontier = SortedSet([first_node])
    explored = set([first_node])

    while len(frontier) != 0:
        curr_node = frontier.pop()
        curr_state = curr_node.get_numpy_state()
        
        for action in Rubik.actions:
            child_state = Rubik.apply_action(curr_state, action)
            if Rubik.is_terminal(child_state):
                actions = [action]
                while curr_node.parent is not None:
                    actions.append(curr_node.action)
                    curr_node = curr_node.parent
                actions.reverse()
                return actions

            child_node = Node(
                child_state,
                steps_taken=curr_node.steps_taken + 1,
                parent=curr_node,
                action=action,
            )
            if child_node in explored:
                continue
            
            frontier.add(child_node)
            explored.add(child_node)

In [13]:
test_actions = solve(np.array([
    [[0, 0, 5],
     [0, 0, 5],
     [0, 0, 5]],

    [[1, 1, 1],
     [1, 1, 1],
     [1, 1, 1]],

    [[2, 2, 0],
     [2, 2, 0],
     [2, 2, 0]],

    [[3, 3, 3],
     [3, 3, 3],
     [3, 3, 3]],

    [[4, 4, 2],
     [4, 4, 2],
     [4, 4, 2]],

    [[5, 5, 4],
     [5, 5, 4],
     [5, 5, 4]],
]))
assert test_actions == ["R"]

In [14]:
test_actions = solve(np.array([
    [[5, 0, 0],
     [5, 0, 0],
     [5, 0, 0]],

    [[1, 1, 1],
     [1, 1, 1],
     [1, 1, 1]],

    [[0, 2, 2],
     [0, 2, 2],
     [0, 2, 2]],

    [[3, 3, 3],
     [3, 3, 3],
     [3, 3, 3]],

    [[2, 4, 4],
     [2, 4, 4],
     [2, 4, 4]],

    [[4, 5, 5],
     [4, 5, 5],
     [4, 5, 5]],
]))
assert test_actions == ["L'"]

In [15]:
test_actions = solve(np.array([
    [[0, 0, 0],
     [0, 0, 0],
     [5, 5, 5]],

    [[2, 2, 0],
     [1, 1, 1],
     [1, 1, 1]],

    [[3, 3, 3],
     [2, 2, 0],
     [2, 2, 0]],

    [[4, 5, 5],
     [3, 3, 3],
     [3, 3, 3]],

    [[4, 4, 2],
     [4, 4, 2],
     [4, 4, 2]],

    [[5, 5, 4],
     [5, 5, 4],
     [1, 1, 1]],
]))
assert test_actions == ["U'", "R"]

In [16]:
test_actions = solve(np.array([
    [[5, 0, 0],
     [5, 0, 0],
     [1, 5, 5]],

    [[1, 1, 2],
     [1, 1, 2],
     [4, 5, 4]],

    [[0, 3, 3],
     [0, 2, 0],
     [1, 1, 0]],

    [[4, 5, 5],
     [3, 3, 3],
     [5, 2, 0]],

    [[2, 2, 3],
     [4, 4, 4],
     [2, 2, 2]],

    [[3, 3, 3],
     [4, 5, 4],
     [4, 1, 1]],
]))
assert test_actions == ["D'",  "L'", "U'", "R"]

Below are some test codes to test the performance of the `solve` function.
Ideally, any cube should be solvable within 20 moves.
Runtime should be reasonable, should be less than 1 minute, ideally in a few seconds.
We test incrementally in difficulty, starting with 7-move cube.

In [7]:
from time import time

def test(cube, model_solution):
    start_time = time()
    print("model solution: ", model_solution)
    actions = solve(cube)
    print("solution found: ", actions)
    print("time taken: ", time() - start_time)
    final_cube = cube
    for action in actions:
        final_cube = Rubik.apply_action(final_cube, action)
    assert Rubik.is_terminal(final_cube)

In [8]:
test(
    np.array([
        [[2, 1, 1],
         [0, 0, 5],
         [2, 0, 1]],
    
        [[4, 1, 4],
         [4, 1, 2],
         [0, 3, 0]],
    
        [[3, 3, 2],
         [0, 2, 0],
         [5, 3, 1]],
    
        [[0, 3, 0],
         [5, 3, 2],
         [4, 4, 4]],
    
        [[3, 2, 5],
         [4, 4, 5],
         [3, 4, 5]],
    
        [[2, 2, 3],
         [1, 5, 1],
         [1, 5, 5]],
    ]),
    ["D", "B", "R'", "L", "U'", "R", "D"],
)

model solution:  ['D', 'B', "R'", 'L', "U'", 'R', 'D']
solution found:  ['D', 'B', 'R', "L'", "U'", 'L', 'D']
time taken:  0.04499697685241699


In [9]:
test(
    np.array([
        [[5, 5, 1],
         [3, 0, 4],
         [4, 5, 1]],

        [[4, 2, 2],
         [5, 1, 0],
         [3, 0, 0]],

        [[1, 1, 2],
         [5, 2, 1],
         [3, 4, 5]],

        [[0, 3, 0],
         [0, 3, 2],
         [3, 4, 4]],

        [[2, 1, 0],
         [3, 4, 2],
         [4, 2, 1]],

        [[2, 0, 5],
         [4, 5, 1],
         [3, 3, 5]],
    ]),
    ["D", "R", "U", "F", "L'", "D'", "R", "U'", "U'", "F"],
)

model solution:  ['D', 'R', 'U', 'F', "L'", "D'", 'R', "U'", "U'", 'F']
solution found:  ['D', 'R', 'U', 'F', "L'", "D'", 'R', "U'", "U'", 'F']
time taken:  0.846031904220581


In [10]:
test(
    np.array([
        [[3, 0, 4],
         [2, 0, 5],
         [5, 1, 3]],

        [[4, 4, 1],
         [1, 1, 4],
         [5, 5, 5]],

        [[0, 2, 4],
         [5, 2, 3],
         [1, 0, 1]],

        [[5, 0, 2],
         [2, 3, 4],
         [0, 0, 2]],

        [[4, 1, 2],
         [3, 4, 3],
         [3, 1, 3]],

        [[0, 5, 0],
         [4, 5, 3],
         [2, 2, 1]],
    ]),
    ["D", "B", "R", "F'", "L", "L", "U", "B", "R", "D", "D", "R", "F"],
)

model solution:  ['D', 'B', 'R', "F'", 'L', 'L', 'U', 'B', 'R', 'D', 'D', 'R', 'F']
solution found:  ['D', 'B', 'R', "F'", "L'", "L'", 'U', 'B', 'R', "D'", "D'", 'R', 'F']
time taken:  4.984026908874512


In [3]:
test(
    np.array([
        [[3, 3, 5],
         [1, 0, 0],
         [4, 1, 4]],

        [[2, 5, 2],
         [5, 1, 2],
         [1, 0, 2]],

        [[1, 0, 5],
         [3, 2, 0],
         [3, 2, 3]],

        [[3, 3, 1],
         [5, 3, 5],
         [0, 1, 0]],

        [[4, 1, 5],
         [2, 4, 4],
         [4, 2, 1]],

        [[5, 4, 2],
         [3, 5, 4],
         [0, 4, 0]],
    ]),
    ["R", "R", "U", "B", "D'", "L'", "F", "U", "F", "R", "B'", "L", "D'", "R", "R", "U'"],
)

From the above test, the coefficient of number of steps taken is adjusted to obtain smaller search time.
For a 13-move puzzle, it takes around 4-5 seconds.
However, 16-move puzzle still takes a very long time.

We therefore automate a search for an optimal coefficient.
For values between 3 - 5 (inclusive), with step 0.1, we do a search on 13-move puzzle and measure the time taken to obtain an answer.
We will terminate the search early if the search takes more than 5 seconds.
A graph of search time against coefficient is plot to visualise the results.

Initially, values of 3 - 5 were searched.

![3-5 plot](images/3-5-plot.png)

Output: [3-5 output](files/3-5-output.txt)

However, it is found that values >= 4 take too long to return a solution (time limit of 5s exceeded). Hence, values between 1 - 4 are searched instead.

We obtained the following:

![1-4 plot](images/1-4-plot.png)

Output: [1-4 output](files/1-4-output.txt)

Optimal values are found between 1.7 - 3.5. We increase time limit to 10s to see the difference more clearly.

![1.7-3.5 plot](images/1.7-3.5-plot.png)

Output: [1.7-3.5 output](files/1.7-3.5-output.txt)

In [3]:
from time import time
import matplotlib.pyplot as plt
from random import randint

In [ ]:
def search_for_coeff():
    time_limit = 10
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken, coeff):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - coeff * steps_taken

    class Node:
        def __init__(self, state, coeff, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken, coeff)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, coeff, start_time):
        first_node = Node(state, coeff)
        frontier = SortedSet([first_node])
        explored = set([first_node])
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return actions, time() - start_time
    
                child_node = Node(
                    child_state,
                    coeff,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                if child_node in explored:
                    continue
                
                frontier.add(child_node)
                explored.add(child_node)

            if time() - start_time > time_limit:
                return None, time_limit

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    coeffs = np.arange(1.7, 3.5, 0.1)
    time_records = []
    for coeff in coeffs:
        time_record = []
        for puzzle, solution in puzzles:
            print(f"Solving puzzle with solution {solution} for coeff {coeff}")
            actual_solution, time_taken = solve(puzzle, coeff, time())
            print(f"Time taken: {time_taken}, solution: {actual_solution}")
            time_record.append(time_taken)
        time_records.append(time_record)

    time_records = np.array(time_records)
    avg_time = np.average(time_records, axis=1)
    deviation = np.max(np.abs(time_records - avg_time[:, None]), axis=1)

    plt.errorbar(coeffs, avg_time, yerr=deviation)

search_for_coeff()

Values between 2.5 - 3.0 seems to be optimal and do not make a difference.
We pick one value in between, 2.8, to see if it can solve any puzzle in a reasonable amount of time.

In [5]:
def test_coeff():
    time_limit = 60
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - 2.8 * steps_taken

    class Node:
        def __init__(self, state, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, start_time):
        first_node = Node(state)
        frontier = SortedSet([first_node])
        explored = set([first_node])
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return time() - start_time, actions
    
                child_node = Node(
                    child_state,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                if child_node in explored:
                    continue
                
                frontier.add(child_node)
                explored.add(child_node)

            if time() - start_time > time_limit:
                return time_limit, None

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    for puzzle, solution in puzzles:
        start_time = time()
        print(f"Solving puzzle with solution {solution}")
        time_taken, actual_solution = solve(puzzle, start_time)
        print(f"Time taken: {time_taken}, solution: {actual_solution}")

test_coeff()

Solving puzzle with solution ["U'", 'L', "B'", 'R', "F'", 'D', 'L', "B'", "D'", "D'", "F'", "R'"]
Time taken: 1.9847972393035889, solution: ["U'", 'L', "B'", 'R', "F'", 'D', 'L', "B'", 'D', 'D', "F'", "R'"]
Solving puzzle with solution ["R'", "R'", "B'", "D'", 'F', "L'", 'D', 'L', "U'", "U'", "R'", "F'"]
Time taken: 1.6053752899169922, solution: ['R', 'R', "B'", "D'", 'F', "L'", 'D', 'L', 'U', 'U', "R'", "F'"]
Solving puzzle with solution ["F'", "L'", 'B', 'B', 'R', "D'", "D'", "F'", 'R', "U'", "L'", "L'"]
Time taken: 5.415637969970703, solution: ["F'", "L'", 'B', 'B', 'R', "D'", "D'", "F'", 'R', "U'", "L'", "L'"]
Solving puzzle with solution ["U'", "B'", 'L', "B'", 'D', "R'", "F'", "F'", "R'", "D'", 'F', "L'"]
Time taken: 0.1093602180480957, solution: ["U'", "B'", 'L', "B'", 'D', "R'", "F'", "F'", "R'", "D'", 'F', "L'"]
Solving puzzle with solution ['U', "R'", 'F', "U'", "U'", 'L', "B'", "U'", "R'", "D'", "D'", "F'"]
Time taken: 12.533198595046997, solution: ['U', "R'", 'F', 'U', 'U',

At this stage, I realised that the fact that a state not being visited again might cause the algorithm to return a longer solution.
This is because the second visit might have taken a lower number of moves.

I hence test out tree search approach, where we do not keep track of explored states.
We keep the same heuristic and do a search for optimal value.

![1.5-4.5 plot](images/tree-1.5-4.5-plot.png)

Output: [1.5-4.5 output](files/tree-1.5-4.5-output.txt)

We found the optimal value falling between 2.5 - 3.2. However, there are still puzzles that run > 10s.
We test the coefficient of 3.0.

In [ ]:
def search_for_coeff():
    time_limit = 10
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken, coeff):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - coeff * steps_taken

    class Node:
        def __init__(self, state, coeff, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken, coeff)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, coeff, start_time):
        first_node = Node(state, coeff)
        frontier = SortedSet([first_node])
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return actions, time() - start_time
    
                child_node = Node(
                    child_state,
                    coeff,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                
                frontier.add(child_node)

            if time() - start_time > time_limit:
                return None, time_limit

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        # ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        # ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        # ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        # ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    coeffs = np.arange(1.5, 4.6, 0.1)
    time_records = []
    for coeff in coeffs:
        time_record = []
        for puzzle, solution in puzzles:
            print(f"Solving puzzle with solution {solution} for coeff {coeff}")
            actual_solution, time_taken = solve(puzzle, coeff, time())
            print(f"Time taken: {time_taken}, solution: {actual_solution}")
            time_record.append(time_taken)
        time_records.append(time_record)

    time_records = np.array(time_records)
    avg_time = np.average(time_records, axis=1)
    deviation = np.max(np.abs(time_records - avg_time[:, None]), axis=1)

    plt.errorbar(coeffs, avg_time, yerr=deviation)

search_for_coeff()

We test the coefficient of 3.0s, increasing the time limit to 60s, and test on all puzzles.

In [4]:
def test_coeff():
    time_limit = 60
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - 3 * steps_taken

    class Node:
        def __init__(self, state, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, start_time):
        first_node = Node(state)
        frontier = SortedSet([first_node])
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return time() - start_time, actions
    
                child_node = Node(
                    child_state,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                
                frontier.add(child_node)

            if time() - start_time > time_limit:
                return time_limit, None

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    for puzzle, solution in puzzles:
        start_time = time()
        print(f"Solving puzzle with solution {solution}")
        time_taken, actual_solution = solve(puzzle, start_time)
        print(f"Time taken: {time_taken}, solution: {actual_solution}")

test_coeff()

Solving puzzle with solution ["U'", 'L', "B'", 'R', "F'", 'D', 'L', "B'", "D'", "D'", "F'", "R'"]
Time taken: 4.166831970214844, solution: ["U'", 'L', "B'", 'R', "F'", 'D', 'L', "B'", 'D', 'D', "F'", "R'"]
Solving puzzle with solution ["R'", "R'", "B'", "D'", 'F', "L'", 'D', 'L', "U'", "U'", "R'", "F'"]
Time taken: 2.0695033073425293, solution: ['R', 'R', "B'", "D'", 'F', "L'", 'D', 'L', 'U', 'U', "R'", "F'"]
Solving puzzle with solution ["F'", "L'", 'B', 'B', 'R', "D'", "D'", "F'", 'R', "U'", "L'", "L'"]
Time taken: 9.641487121582031, solution: ["F'", "L'", 'B', 'B', 'R', "D'", "D'", "F'", 'R', "U'", "L'", "L'"]
Solving puzzle with solution ["U'", "B'", 'L', "B'", 'D', "R'", "F'", "F'", "R'", "D'", 'F', "L'"]
Time taken: 0.2546858787536621, solution: ["U'", "B'", 'L', "B'", 'D', "R'", "F'", "F'", "R'", "D'", 'F', "L'"]
Solving puzzle with solution ['U', "R'", 'F', "U'", "U'", 'L', "B'", "U'", "R'", "D'", "D'", "F'"]
Time taken: 20.424864530563354, solution: ['U', "R'", 'F', 'U', 'U', 

Compared to the graph search approach, tree search performs roughly equally on every puzzle,
and for any puzzle, if graph search cannot solve within 60s, tree search also cannot solve within 60s.
There are a few approaches moving forward:

* Improve the tracking of explored states, so that we allow second visit to a state with fewer number of moves. We probably can make use of a hashmap to map each state to the number of moves, instead of a hashset.
* Improve the heuristic. Basically make a new heuristic, since finding the optimal coefficient still does not produce a good result.
* Develop a neural network, with input being the state and output being the one optimal move for the state. This requires synthesis of data

We try first approach, which is to use a hashmap to store the list of explored states. We use the coefficient of 3.0, a near-optimal coefficient.

In [5]:
def test_coeff():
    time_limit = 60
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - 3 * steps_taken

    class Node:
        def __init__(self, state, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, start_time):
        first_node = Node(state)
        frontier = SortedSet([first_node])
        explored = {
            first_node: first_node.steps_taken,
        }
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return time() - start_time, actions
    
                child_node = Node(
                    child_state,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                if child_node in explored:
                    if child_node.steps_taken < explored[child_node]:
                        explored[child_node] = child_node.steps_taken
                        frontier.add(child_node)
                else:
                    explored[child_node] = child_node.steps_taken
                    frontier.add(child_node)

            if time() - start_time > time_limit:
                return time_limit, None

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    for puzzle, solution in puzzles:
        start_time = time()
        print(f"Solving puzzle with solution {solution}")
        time_taken, actual_solution = solve(puzzle, start_time)
        print(f"Time taken: {time_taken}, solution: {actual_solution}")

test_coeff()

Solving puzzle with solution ["U'", 'L', "B'", 'R', "F'", 'D', 'L', "B'", "D'", "D'", "F'", "R'"]
Time taken: 2.6950454711914062, solution: ["U'", 'L', "B'", 'R', "F'", 'D', 'L', "B'", 'D', 'D', "F'", "R'"]
Solving puzzle with solution ["R'", "R'", "B'", "D'", 'F', "L'", 'D', 'L', "U'", "U'", "R'", "F'"]
Time taken: 1.5013577938079834, solution: ['R', 'R', "B'", "D'", 'F', "L'", 'D', 'L', 'U', 'U', "R'", "F'"]
Solving puzzle with solution ["F'", "L'", 'B', 'B', 'R', "D'", "D'", "F'", 'R', "U'", "L'", "L'"]
Time taken: 6.0894455909729, solution: ["F'", "L'", 'B', 'B', 'R', "D'", "D'", "F'", 'R', "U'", "L'", "L'"]
Solving puzzle with solution ["U'", "B'", 'L', "B'", 'D', "R'", "F'", "F'", "R'", "D'", 'F', "L'"]
Time taken: 0.12498307228088379, solution: ["U'", "B'", 'L', "B'", 'D', "R'", "F'", "F'", "R'", "D'", 'F', "L'"]
Solving puzzle with solution ['U', "R'", 'F', "U'", "U'", 'L', "B'", "U'", "R'", "D'", "D'", "F'"]
Time taken: 11.904851198196411, solution: ['U', "R'", 'F', 'U', 'U', 

Last 3 puzzles are still not solvable within reasonable time. 
There is no significant difference in performance compared to normal graph or tree search.

We now experiment with a few different heuristic.
I speculate that the main reason for not solving the puzzle fast enough is because it does not sufficiently explore the first few moves.
The algorithm may be stuck with the first move that significantly improve the heuristic value, but may in long term produces a longer solution.

We experiment with varying emphasis on the uniformity.

We test with hashmap approach first. If it successfully solve the difficult puzzles in a short period of time, graph and tree search are explored to optimise the search.

<hr>

* Hashmap
* Heuristic: `(26 + steps_taken) / 52 * uniformity - coeff * steps_taken`
* Range: `1.5 - 4.5`

Output: [1.5 - 4.5 output](files/hashmap-1.5-4.5-inc-output.txt)

This variant performs slightly worse than the original heuristic.

![1.5 - 4.5 plot](images/hashmap-1.5-4.5-inc-plot.png)

<hr>

* Hashmap
* Heuristic: `(52 - steps_taken) / 52 * uniformity - coeff * steps_taken`
* Range: `1.5 - 4.5`

Output: [1.5 - 4.5 output](files/hashmap-1.5-4.5-desc-output.txt)

This variant performs slightly better than the original heuristic.
However, it still cannot solve the more challenging puzzles within 10s.

![1.5 - 4.5 plot](images/hashmap-1.5-4.5-desc-plot.png)

<hr>

* Hashmap
* Heuristic: `uniformity - coeff * steps_taken ^ 2`
* Range: 0.0 - 3.0

Output: [0.0 - 3.0 output](files/hashmap-0.0-3.0-square-output.txt)

Unsurprisingly, only small coefficients return some solution in time.

* Range: 0.0 - 0.4

Output: [0.0 - 0.4 output](files/hashmap-0.0-0.4-square-output.txt)

An interesting observation is that this version with `coeff == 0`
managed to solve one of the difficult puzzles that the other heuristic could not,
but cannot solve normal puzzles efficiently.
The solution returned is not the shortest solution.

![0.0 - 0.4 plot](images/hashmap-0.0-0.4-square-plot.png)

In [ ]:
def search_for_coeff():
    time_limit = 10
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken, coeff):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - coeff * steps_taken ** 2

    class Node:
        def __init__(self, state, coeff, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken, coeff)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, coeff, start_time):
        first_node = Node(state, coeff)
        frontier = SortedSet([first_node])
        explored = {
            first_node: first_node.steps_taken,
        }
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return actions, time() - start_time
    
                child_node = Node(
                    child_state,
                    coeff,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                if child_node in explored:
                    if explored[child_node] > child_node.steps_taken:
                        explored[child_node] = child_node.steps_taken
                        frontier.add(child_node)
                else:
                    explored[child_node] = child_node.steps_taken
                    frontier.add(child_node)

            if time() - start_time > time_limit:
                return None, time_limit

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        # ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        # ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        # ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        # ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    coeffs = np.arange(0.0, 0.4, 0.01)
    time_records = []
    for coeff in coeffs:
        time_record = []
        for puzzle, solution in puzzles:
            print(f"Solving puzzle with solution {solution} for coeff {coeff}")
            actual_solution, time_taken = solve(puzzle, coeff, time())
            print(f"Time taken: {time_taken}, solution: {actual_solution}")
            time_record.append(time_taken)
        time_records.append(time_record)

    time_records = np.array(time_records)
    avg_time = np.average(time_records, axis=1)
    deviation = np.max(np.abs(time_records - avg_time[:, None]), axis=1)

    plt.errorbar(coeffs, avg_time, yerr=deviation)

search_for_coeff()

From the last result above, it is speculated that
the algorithm may spend too much time exploring beyond 26 moves.
We now use research's result to depth-limit the search,
not exploring anything beyond 26 moves.

* Hashmap
* Heuristic: uniformity - coeff * steps_taken
* Range: 1.5 - 4.5

Output: [1.5 - 4.5 output](files/hashmap-1.5-4.5-limit-output.txt)

There is no significant improvement in search time. Notably, it still cannot solve the more challenging puzzles.

![1.5 - 4.5 plot](images/hashmap-1.5-4.5-limit-plot.png)

In [ ]:
def search_for_coeff():
    time_limit = 10
    step_limit = 26
    
    # redefine the functions and classes to be used, except from `Rubik` class.
    def heuristic(state, steps_taken, coeff):
        def face_heuristic(face):
            count = 0
            for i in range(3):
                for j in range(3):
                    if i < 2 and face[i][j] == face[i + 1][j]:
                        count += 1
                    if j < 2 and face[i][j] == face[i][j + 1]:
                        count += 1
            return count
        return sum([face_heuristic(face) for face in state]) - coeff * steps_taken

    class Node:
        def __init__(self, state, coeff, steps_taken=0, parent=None, action=None):
            self.state = Rubik.turn_tuple(state)
            self.steps_taken = steps_taken
            self.parent = parent
            self.action = action
            self.h_value = heuristic(state, steps_taken, coeff)
    
        def __lt__(self, node):
            return self.h_value < node.h_value
    
        def __hash__(self):
            return hash(self.state)
    
        def __eq__(self, other):
            return self.state == other.state
            
        def get_numpy_state(self):
            return Rubik.turn_numpy(self.state)

    def solve(state, coeff, start_time):
        first_node = Node(state, coeff)
        frontier = SortedSet([first_node])
        explored = {
            first_node: first_node.steps_taken,
        }
    
        while len(frontier) != 0:
            curr_node = frontier.pop()
            curr_state = curr_node.get_numpy_state()
            
            for action in Rubik.actions:
                child_state = Rubik.apply_action(curr_state, action)
                if Rubik.is_terminal(child_state):
                    actions = [action]
                    while curr_node.parent is not None:
                        actions.append(curr_node.action)
                        curr_node = curr_node.parent
                    actions.reverse()
                    return actions, time() - start_time

                if curr_node.steps_taken == step_limit - 1:
                    continue
    
                child_node = Node(
                    child_state,
                    coeff,
                    steps_taken=curr_node.steps_taken + 1,
                    parent=curr_node,
                    action=action,
                )
                if child_node in explored:
                    if explored[child_node] > child_node.steps_taken:
                        explored[child_node] = child_node.steps_taken
                        frontier.add(child_node)
                else:
                    explored[child_node] = child_node.steps_taken
                    frontier.add(child_node)

            if time() - start_time > time_limit:
                return None, time_limit

    # define util functions
    def reverse_move(move):
        if len(move) == 2:
            return move[0]
        else:
            return move + "'"

    def get_terminal_state():
        return np.array([
            [[i for k in range(3)] for j in range(3)] for i in range(6)
        ])

    def generate_puzzle(scramble):
        puzzle = get_terminal_state()
        for action in scramble:
            puzzle = Rubik.apply_action(puzzle, action)
        scramble.reverse()
        solution = [reverse_move(move) for move in scramble]
        return puzzle, solution

    scrambles = [
        # ["R", "F", "D", "D", "B", "L'", "D'", "F", "R'", "B", "L'", "U"],
        ["F", "R", "U", "U", "L'", "D'", "L", "F'", "D", "B", "R", "R"],
        ["L", "L", "U", "R'", "F", "D", "D", "R'", "B'", "B'", "L", "F"],
        ["L", "F'", "D", "R", "F", "F", "R", "D'", "B", "L'", "B", "U"],
        # ["F", "D", "D", "R", "U", "B", "L'", "U", "U", "F'", "R", "U'"],
        ["F'", "R", "D", "D", "F'", "U", "L", "B", "L", "U'", "F", "R"],
        # ["R", "F", "U", "R'", "D'", "L'", "U'", "R", "U", "R", "U", "F"],
        ["R", "U", "U", "L'", "F", "R", "F", "L'", "D", "B", "B", "D'"],
        ["L", "U", "F'", "R", "D", "D", "L", "U'", "L'", "B", "R", "D"],
        ["U", "F", "D'", "R'", "B", "L", "D", "L", "F", "B", "R", "R"],
        # ["R", "F", "D", "B", "L", "F'", "R'", "U", "L", "B", "F", "R"],
    ]
    puzzles = [generate_puzzle(scramble) for scramble in scrambles]
    coeffs = np.arange(1.5, 4.5, 0.1)
    time_records = []
    for coeff in coeffs:
        time_record = []
        for puzzle, solution in puzzles:
            print(f"Solving puzzle with solution {solution} for coeff {coeff}")
            actual_solution, time_taken = solve(puzzle, coeff, time())
            print(f"Time taken: {time_taken}, solution: {actual_solution}")
            time_record.append(time_taken)
        time_records.append(time_record)

    time_records = np.array(time_records)
    avg_time = np.average(time_records, axis=1)
    deviation = np.max(np.abs(time_records - avg_time[:, None]), axis=1)

    plt.errorbar(coeffs, avg_time, yerr=deviation)

search_for_coeff()